In [ ]:
from istio_auth_with_client import client

In [2]:
import kfp

https://www.kubeflow.org/docs/components/pipelines/v1/sdk/manipulate-resources/

시키는데로 하고 있기는한데... 설명이 아무것도 없고 어디서 import 하는지도 안적혀 있어서... 

kubeflow document에서 k8s client 쓰는걸 보여주면 어디서 import하는지 설치하는지 정도는 나와야 하는거 아닌가...  

설명 하는 블로그도 있는데 하나같이 다 python은 아님
https://recordsoflife.tistory.com/1339

https://stackoverflow.com/questions/56673919/kubernetes-python-api-client-execute-full-yaml-file

쉐에엣! 감사합니다!

In [17]:
from kubernetes import client as k8s_client
from kubernetes import config, utils

In [18]:
#config.load_kube_config()

In [19]:
k8s_api_client = k8s_client.ApiClient()

In [20]:
yaml_file = './spark-job-python.yaml' #'<location to your multi-resource file>'

In [21]:
utils.create_from_yaml(k8s_api_client, yaml_file)
#k8s_client.V1Job("spark-job-python.yaml")

AttributeError: module 'kubernetes.client' has no attribute 'SparkoperatorV1beta2Api'

## AttributeError: module 'kubernetes.client' has no attribute 'SparkoperatorV1beta2Api'
## ㅈ 된거 같은데? 지원을 안해서 못쓰는거 같은데?
### 마침 나랑 같은 문제가 있던 사람이 있는듯

https://github.com/GoogleCloudPlatform/spark-on-k8s-operator/issues/1677

이 방법은 custom resource용이 아니라고 함.

https://github.com/kubernetes-client/python/blob/master/kubernetes/docs/CustomObjectsApi.md

이걸 보라네요.  

공식이랑 깃허브 이슈랑 적절히 합쳐서 보면 간신히 보임  

거기에 yaml파일을 json object로 만들어서 써라 라고만 적혀있고 쓰는 법은 없는데  
이건 기존 pipeline 에서 yaml 라이브러리를 쓰면 될거 같아.

In [2]:
from kubernetes import client as k8s_client

configuration = k8s_client.Configuration()
#configuration.api_key['authorization'] = 'YOUR_API_KEY' #내 키? 그런게 있나?

NameError: name 'k8s_config' is not defined

In [1]:
from kubernetes import client as k8s_client

from kubernetes import config
config.load_incluster_config()

k8s_config = k8s_client.Configuration()
#k8s_config.host = "https://10.100.0.1"
print(k8s_config.host)
k8s_config.host = "k8s://https://kubernetes.default.svc:443"
#"172.31.12.123:8889"
print(k8s_config.host)
k8s_config.verify_ssl = False

k8s_api_client = k8s_client.ApiClient(k8s_config)
k8s_spark_api_client = k8s_client.CustomObjectsApi(k8s_api_client)
#k8s_spark_api_client = k8s_client.CustomObjectsApi()

k8s_spark_properties = dict(
    group = "sparkoperator.k8s.io",
    version = "v1beta2",
    plural = "sparkapplication",
    namespace = "kubeflow-user-example-com",
    #serviceaccount = "my-release-spark"
)



https://10.100.0.1:443
k8s://https://kubernetes.default.svc:443


In [2]:
#load job yaml to application_config variable
import yaml
with open("spark-job-python.yaml", "r") as stream:
    spark_job_manifest = yaml.safe_load(stream)

application_config = spark_job_manifest

In [3]:
k8s_spark_api_client.create_namespaced_custom_object(
    body=application_config, **k8s_spark_properties
)

URLSchemeUnknown: Not supported URL scheme k8s

갑자기 현타옴 ㅋㅋㅋㅋ 이거 그냥 resource를 손으로 적으면 해결되는 건데    

어?  

책 보니까 source yaml 파일인거 json으로 resource 변수에 선언하고 바로 resource op에 넣으면 되는데?  

ㅋㅋㅋㅋㅋ 무슨 뻘짓을 한거지?


"status":"Failure","message":"sparkapplication.sparkoperator.k8s.io is forbidden: User \"system:serviceaccount:kubeflow-user-example-com:default-editor\" cannot create resource \"sparkapplication\" in API group \"sparkoperator.k8s.io\" in the namespace \"kubeflow-user-example-com\""  

이 에러와 유사한 예시가 kfserving에 있어서 가져와봄... 도움은 안됨
https://stackoverflow.com/questions/72474007/kubeflow-forbidden-user-systemserviceaccountkubeflow-user-example-comdefault

https://github.com/kubernetes-client/python/issues/254

api서버가 아니라 localhost에 보내려고 해서 생기는 문제다.



ubuntu@ip-172-31-12-123:~$ kubectl config view
apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: DATA+OMITTED
    server: https://7B8222FBA4B0D6AECE9B8BB3F03A4D19.gr7.us-east-1.eks.amazonaws.com
  name: arn:aws:eks:us-east-1:222170749288:cluster/eks-cluster
contexts:
- context:
    cluster: arn:aws:eks:us-east-1:222170749288:cluster/eks-cluster
    user: arn:aws:eks:us-east-1:222170749288:cluster/eks-cluster
  name: arn:aws:eks:us-east-1:222170749288:cluster/eks-cluster
current-context: arn:aws:eks:us-east-1:222170749288:cluster/eks-cluster
kind: Config
preferences: {}
users:
- name: arn:aws:eks:us-east-1:222170749288:cluster/eks-cluster
  user:
    exec:
      apiVersion: client.authentication.k8s.io/v1alpha1
      args:
      - --region
      - us-east-1
      - eks
      - get-token
      - --cluster-name
      - eks-cluster
      command: aws
      env: null
      interactiveMode: IfAvailable
      provideClusterInfo: false


이게 더 맞는 표현인듯  


kubectl proxy --port=8888

 curl http://localhost:8888/api/
{
  "kind": "APIVersions",
  "versions": [
    "v1"
  ],
  "serverAddressByClientCIDRs": [
    {
      "clientCIDR": "0.0.0.0/0",
      "serverAddress": "ip-10-0-120-0.ec2.internal:443"
    }
  ]
}

여기엔 이렇게 적혀있음

kubectl cluster-info
Kubernetes control plane is running at https://7B8222FBA4B0D6AECE9B8BB3F03A4D19.gr7.us-east-1.eks.amazonaws.com
CoreDNS is running at https://7B8222FBA4B0D6AECE9B8BB3F03A4D19.gr7.us-east-1.eks.amazonaws.com/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

https://coffeewhale.com/apiserver

블로그에서는 control plan 위치를 api server라고 보는 듯

이거 하면 나온다고 함.
export KUBECONFIG=/my/path/kubeconfig
cat $HOME/.kube/config

##api server 주소 제대로 적용된 config 쓰기
https://github.com/kubernetes-client/python/issues/254
    
jupyter랑 그냥 .py랑 번갈아가면서 쓰니까. 서로 다른 에러 로그가 나와서 비교하기 좋음  

kubernetes.config.config_exception.ConfigException: Service host/port is not set.  

.py는 그냥 config만 안불러지는거 같아서 jupyter에서 찾은 주소를 하드코딩으로 넣어줌

그랬더니 다 읽고 멈추는데?
=> 보니까 못 불러와서 하루종일 대기하고 있는 듯 => 완료 print가 안됨

In [30]:
spark_job_manifest

{'apiVersion': 'sparkoperator.k8s.io/v1beta2',
 'kind': 'SparkApplication',
 'metadata': {'name': 'pyspark-pi', 'namespace': 'kubeflow-user-example-com'},
 'spec': {'type': 'Python',
  'pythonVersion': '3',
  'mode': 'cluster',
  'image': 'gcr.io/spark-operator/spark-py:v3.1.1',
  'imagePullPolicy': 'Always',
  'mainApplicationFile': 'local:///opt/spark/examples/src/main/python/pi.py',
  'sparkVersion': '3.1.1',
  'restartPolicy': {'type': 'OnFailure',
   'onFailureRetries': 3,
   'onFailureRetryInterval': 10,
   'onSubmissionFailureRetries': 5,
   'onSubmissionFailureRetryInterval': 20},
  'driver': {'cores': 1,
   'coreLimit': '1200m',
   'memory': '512m',
   'labels': {'version': '3.1.1'},
   'serviceAccount': 'my-release-spark'},
  'executor': {'cores': 1,
   'instances': 1,
   'memory': '512m',
   'labels': {'version': '3.1.1'}}}}

{'experiments': [{'created_at': datetime.datetime(2023, 7, 22, 15, 16, 47, tzinfo=tzlocal()),
                  'description': None,
                  'id': '82029175-a46b-4452-a0be-dc5066258bfa',
                  'name': 'test',
                  'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'}],
 'next_page_token': None,
 'total_size': 1}


In [ ]:
from kfp import compiler
exp = client.create_experiment(name='mdupdate2')
print(client.list_experiments())

exp = client.get_experiment(experiment_name = "mdupdate")

compiler.Compiler().compile(recommender_pipeline, "pipeline.tar.gz")

run = client.run_pipeline(exp.id, "pipeline1", "pipeline.tar.gz")



In [ ]:

client.create_run_from_pipeline_func(conditional_pipeline, arguments={})



In [1]:
import kfp
import kfp.dsl as dsl

In [11]:
#load job yaml to application_config variable
import yaml
with open("spark-job-python.yaml", "r") as stream:
    spark_job_manifest = yaml.safe_load(stream)

@dsl.pipeline(name="local_pipeline", description="no_need_to_ask_why")
def local_pipeline():
    rop = dsl.ResourceOp(
        name="foo",
        k8s_resource=spark_job_manifest,#resource,
        #action="create",
        #success_confition="status.applicationState.state == COMPLETED"
    )

In [12]:
from istio_auth_with_client import kfp_client

In [13]:
from kfp import compiler

exp = kfp_client.create_experiment(name='please')
print(kfp_client.list_experiments())

exp = kfp_client.get_experiment(experiment_name="please")

compiler.Compiler().compile(local_pipeline, 'please.yaml')

{'experiments': [{'created_at': datetime.datetime(2023, 7, 22, 15, 16, 47, tzinfo=tzlocal()),
                  'description': None,
                  'id': '82029175-a46b-4452-a0be-dc5066258bfa',
                  'name': 'test',
                  'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'},
                 {'created_at': datetime.datetime(2023, 7, 22, 19, 5, 53, tzinfo=tzlocal()),
                  'description': None,
                  'id': '17a593fd-012e-45aa-ad02-7ac37056f4e0',
                  'name': 'please',
                  'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                                   'type': 'NAMESPACE'},
                              

In [15]:
run = kfp_client.run_pipeline(exp.id, "Go-SPARK!", "please.yaml")

This step is in Failed state with this message: Error (exit code 1): no more retries Error from server (Forbidden): error when creating "/tmp/manifest.yaml": sparkapplications.sparkoperator.k8s.io is forbidden: User "system:serviceaccount:kubeflow-user-example-com:default-editor" cannot create resource "sparkapplications" in API group "sparkoperator.k8s.io" in the namespace "kubeflow-user-example-com"

이놈의 책은 도움이 되나 싶었더니 바로 통수를 치네?

이새끼가 짠 코드는 제대로 돌아가는 꼴을 본적이 없음.

spark operator image가 있네?

https://stackoverflow.com/questions/33322103/multiple-froms-what-it-means
    

https://github.com/kubeflow/kubeflow/issues/5043
    
파이프라인 안쓰고 직접 쓸 수 있냐고 물어보네...